In [58]:
#Se importan los paquetes necesarios para los cálculos.
import pandas as pd #manejo de datos
import numpy as np #parquete de matemática
import scipy.stats as sp #paquete de estadísticas
from sqlalchemy import create_engine #consultas a sql
import sqlalchemy #manejo de sql
import sys #manejo del sistema para recibir argumentos
import warnings #
from sqlalchemy import text
warnings.filterwarnings("ignore")
import pyodbc

import funciones

In [59]:
#Conexión a la base de datos
server = '10.0.0.153'
user = "testipc2"
pas = "Abc$2020"
#user = input('Ingrese el usuario: ')
#pas = input('Ingrese la contraseña: ')
try:
    #driver = 'SQL Server Native Client 11.0'
    driver = 'ODBC Driver 17 for SQL Server'
    database_connection = f'mssql://{user}:{pas}@{server}/master?driver={driver}'
    engine = create_engine(database_connection)
    connection = engine.connect()
    print('Conexión exitosa')
except:
    print('Fallo en la conexión')

Conexión exitosa


In [60]:
#Recibimos el año, mes y década. Deben ingresarse el número de año, mes y década. Las décadas están codificadas de la siguiente forma:
#Todo el mes: década = 0
#Las primeras dos décadas: década = 12
#Las últimas dos décadas: década = 23
#Para cada década individual se ingresa el número de década
#anio = int(input('Ingrese el año: '))
#mes = int(input('Ingrese el mes: '))
#dec = int(input('Ingrese la década (ingrese 0 para obtener todo el mes): '))

In [61]:
anio = 2024
mes = 1
dec = 0

In [62]:
#Se seleccionan los códigos de los artículos, los demás códigos pueden obtenerse a partir de éstos.
def Codigos(connect):
    querycod = text(f"SELECT DISTINCT ArtCod FROM IPC2010_01_RN.dbo.IPCPH6 WHERE PerAno = 2010 AND PerMes = 12 AND ArtCod != 091110301")
    
    return pd.read_sql(querycod, connect, index_col = 'ArtCod')

codigos = Codigos(connection)
#codigos

In [63]:
#Se define una función para hallar cualquier conjunto de códigos.
#le asocia a la categoría su número en la base de datos
di = {'Art':6,'Gba':5,'Sub':4,'Gru':3,'Agr':2,'Div':1}
#le asocia a cada categoría la longitud de su código
leng = {'Div' : 2, 'Agr' : 3, 'Gru' : 4, 'Sub' : 5, 'Gba' : 7, 'Art': 9}

#Se seleccionan los códigos de los artículos, los demás códigos pueden obtenerse a partir de éstos.
def Nombres(datos, cod : str, connect = connection, dict = di):
    ''' 
    Esta función retorna un dataframe con los nombres y códigos de una categoría específica.

    Args:
    cod (DataFrame) : El dataframe con los códigos de una categoría dada, puede ser de artículos, gasto báscico, división, etc
    ind (str) : El código de categoría que deseamos. Gba : Gasto Básico, Sub : Subgrupo, Gru : Grupo, Agr : Agrupación, Div : División
    '''
    if cod == 'Art':
        querycod = text(f"SELECT DISTINCT ArtCod, ArtNom FROM IPC2010_RN.dbo.IPC007")
    else:
        querycod = text(f'SELECT DISTINCT {cod}Cod, {cod}Nom FROM IPC2010_RN.dbo.IPCM0{dict[cod]}')

    nombres = pd.read_sql(querycod, connect, index_col = cod+'Cod')
    return pd.merge(nombres.reset_index(), datos.reset_index(), on = cod+'Cod',  how = 'inner').set_index(['RegCod',cod+'Cod'])

def CodigosCat(ind :str, cod, len = leng):
    ''' 
    Esta función retorna un dataframe con los códigos de una categoría específica.

    Args:
    cod (DataFrame) : El dataframe con los códigos de una categoría dada, puede ser de artículos, gasto báscico, división, etc
    ind (str) : El código de categoría que deseamos. Gba : Gasto Básico, Sub : Subgrupo, Gru : Grupo, Agr : Agrupación, Div : División
    '''
    return pd.DataFrame([x[:len[ind]] for x in cod.index],columns=[ind+'Cod']).drop_duplicates().set_index(ind+'Cod')

In [64]:
#Aquí se mandan a traer las boletas del mes
def Boletas(anio : int, mes : int, connect):

    """Esta función retorna un dataframe con las boletas del mes correspondiente con el índice como . 
    Incluyen código de región, de artículo

    args:
    anio (int): el año de las boletas a traer
    mes (int): el mes de las boletas a traer
    connect (Connection) : conexión a la base de datos

    """
    subquery = text(f"SELECT DISTINCT ArtCod FROM IPC2010_01_RN.dbo.IPCPH6 WHERE PerAno = 2010 AND PerMes = 12 AND ArtCod != 091110301")
    query = text(f"SELECT * FROM (\
            SELECT RegCod, ArtCod, BolNum, ArtPhi,ArtPac,UraChi,UreCan,ArtCR, PerSem\
            FROM IPC2010_01_RN.dbo.IPC104\
            WHERE PerAno = {anio} AND PerMes = {mes} \
            AND ArtCod IN ({subquery})\
            UNION ALL\
            SELECT RegCod, ArtCod, BolNum, ArtPhi,ArtPac,UraChi,UreCan,ArtCR, PerSem\
            FROM IPC2010_02_RN.dbo.IPC104\
            WHERE PerAno = {anio} AND PerMes = {mes} \
            AND ArtCod IN ({subquery})\
            UNION ALL\
            SELECT RegCod, ArtCod, BolNum, ArtPhi,ArtPac,UraChi,UreCan,ArtCR, PerSem\
            FROM IPC2010_03_RN.dbo.IPC104\
            WHERE PerAno = {anio} AND PerMes = {mes} \
            AND ArtCod IN ({subquery})\
            UNION ALL\
            SELECT RegCod, ArtCod, BolNum, ArtPhi,ArtPac,UraChi,UreCan,ArtCR, PerSem\
            FROM IPC2010_04_RN.dbo.IPC104\
            WHERE PerAno = {anio} AND PerMes = {mes} \
            AND ArtCod IN ({subquery})\
            UNION ALL\
            SELECT RegCod, ArtCod, BolNum, ArtPhi,ArtPac,UraChi,UreCan,ArtCR, PerSem\
            FROM IPC2010_05_RN.dbo.IPC104\
            WHERE PerAno = {anio} AND PerMes = {mes} \
            AND ArtCod IN ({subquery})\
            UNION ALL\
            SELECT RegCod, ArtCod, BolNum, ArtPhi,ArtPac,UraChi,UreCan,ArtCR, PerSem\
            FROM IPC2010_06_RN.dbo.IPC104\
            WHERE PerAno = {anio} AND PerMes = {mes} \
            AND ArtCod IN ({subquery})\
            UNION ALL\
            SELECT RegCod, ArtCod, BolNum, ArtPhi,ArtPac,UraChi,UreCan,ArtCR, PerSem\
            FROM IPC2010_07_RN.dbo.IPC104\
            WHERE PerAno = {anio} AND PerMes = {mes} \
            AND ArtCod IN ({subquery})\
            UNION ALL\
            SELECT RegCod, ArtCod, BolNum, ArtPhi,ArtPac,UraChi,UreCan,ArtCR, PerSem\
            FROM IPC2010_08_RN.dbo.IPC104\
            WHERE PerAno = {anio} AND PerMes = {mes} \
            AND ArtCod IN ({subquery})\
            ) I\
            ORDER BY RegCod, ArtCod")
    return pd.read_sql(query, connect, index_col=['RegCod', 'ArtCod', 'BolNum']).sort_index()

boletas = Boletas(anio, mes, connection)

In [65]:
boletas

ArtPhi  ArtPac  UraChi  UreCan ArtCR  PerSem
RegCod ArtCod    BolNum                                               
01     011110101 315960     7.00     0.0   400.0     0.0     N       1
                 315968     7.10     0.0   400.0     0.0     N       1
                 315969     7.10     0.0   400.0     0.0     N       1
                 315970     7.15     0.0   400.0     0.0     N       1
                 315973     7.10     0.0   400.0     0.0     N       1
...                          ...     ...     ...     ...   ...     ...
08     125110301 110950    50.00     0.0     1.0     0.0     N       3
       125110401 110241  2500.00     0.0     1.0     0.0     N       1
                 110326  2000.00     0.0     1.0     0.0     N       1
                 110684  2800.00     0.0     1.0     0.0     N       2
                 110992  4000.00     0.0     1.0     0.0     N       3

[48741 rows x 6 columns]

In [66]:
#aqui se empieza la parte de la conexion con el otro server
#Almacenamos la información del servidor para poder realizar la conexion más adelante
server = 'ipcprod.database.windows.net'
user = 'ipcreader'
password = '1pc/*2023'
database = 'db-indices'


In [67]:
#Se crea un string para realizar la conexión, con la informacion del servidor almacenada previamente
conn_str = (
    f'Driver={{ODBC Driver 17 for SQL Server}};'
    f'Server={server};'
    f'Database={database};'
    f'Uid={user};'
    f'Pwd={password};'
)

In [68]:
try:
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    
    cursor.execute("EXEC [dbo].[sp_get_precios_recolectados_mes] 2024, 1")
    boletas01 = pd.DataFrame([tuple(row) for row in cursor.fetchall()], columns=[column[0] for column in cursor.description])
    
    conn.close()
    print("Connection successful")
except Exception as e:
    print(f"Error: {e}")

Connection successful


In [69]:
boletas02 = boletas01
boletas02['codigo_articulo'] = boletas01['codigo_articulo'].replace(funciones.recodificacion) #esta parte hace la recodificacion con el diccionario de los codigos en el archivo funciones
boletas02['decada'] = boletas02['decada'].replace(funciones.decadas)
boletas02['nt_tipo'] = boletas02['nt_tipo'].replace({351761:'S', 351762:'S'})
boletas02['nt_tipo'] = boletas02['nt_tipo'].fillna('N')
boletas02['cantidad_anterior'] = pd.to_numeric(boletas02['cantidad_anterior'], errors='coerce')
boletas02['cantidad_actual'] = pd.to_numeric(boletas02['cantidad_actual'], errors='coerce')
boletas02['precio_anterior'] = pd.to_numeric(boletas02['precio_anterior'], errors='coerce')
boletas02['precio_actual'] = pd.to_numeric(boletas02['precio_actual'], errors='coerce')
boletas02['decada'] = pd.to_numeric(boletas02['decada'], errors='coerce')
boletas02['region'] = boletas02['region'].astype(str)
boletas02['codigo_articulo'] = boletas02['codigo_articulo'].astype(str)
boletas02 = boletas02[['region', 'codigo_articulo', 'ine_poll_id', 'cantidad_anterior', 'cantidad_actual', 'precio_anterior', 'precio_actual', 'nt_tipo', 'decada']]
boletas02.rename(columns={'region':'RegCod', 'codigo_articulo':'ArtCod', 'ine_poll_id':'BolNum', 'cantidad_anterior':'UraChi', 'cantidad_actual':'UreCan', 'precio_anterior':'ArtPhi', 'precio_actual':'ArtPac', 'nt_tipo':'ArtCR', 'decada':'PerSem'}, inplace=True)
boletas02['RegCod'] = boletas02['RegCod'].replace({'1':'01', '2':'02', '3':'03', '4':'04', '5':'05', '6':'06', '7':'07','8':'08',})

boletas02

,RegCod,ArtCod,BolNum,UraChi,UreCan,ArtPhi,ArtPac,ArtCR,PerSem
0,01,053110202,34-1-2024,1.0,1.0,145.0,145.0,N,1
1,01,031220302,36-1-2024,1.0,1.0,250.0,250.0,N,1
2,01,072110201,42-1-2024,1.0,1.0,325.0,400.0,N,1
3,01,041110103,43-1-2024,1.0,1.0,1700.0,1700.0,N,1
4,01,072110302,54-1-2024,946.0,946.0,70.0,70.0,N,1
...,...,...,...,...,...,...,...,...,...
51522,08,053110101,18268-1-2024,0.0,1.0,0.0,115.0,N,3
51523,08,053110301,18268-1-2024,0.0,1.0,0.0,1.0,N,3
51524,08,055110202,18268-1-2024,0.0,350.0,0.0,7.5,N,3
51525,08,055120401,18268-1-2024,0.0,1.0,0.0,7.0,N,3


In [70]:
boletas = boletas02[boletas02['ArtCod'].str.len() == 9]
boletas.set_index(['RegCod', 'ArtCod', 'BolNum'], inplace=True)
boletas.sort_index(inplace=True)
boletas

UraChi  UreCan  ArtPhi   ArtPac ArtCR  PerSem
RegCod ArtCod    BolNum                                                     
01     011110101 1035-1-2024    400.0   400.0     7.1     7.10     N       1
                 1104-1-2024    400.0   400.0     7.5     7.50     N       2
                 1115-1-2024    400.0   400.0     7.0     7.00     N       1
                 113-1-2024     400.0   400.0     7.0     7.15     N       1
                 1175-1-2024    400.0   400.0     7.1     7.10     N       2
...                               ...     ...     ...      ...   ...     ...
08     125110401 16025-1-2024     1.0     1.0  2500.0  2500.00     N       1
                 16027-1-2024     1.0     1.0  2000.0  2000.00     N       1
                 16052-1-2024     1.0     1.0  3800.0  3800.00     N       3
                 16435-1-2024     1.0     1.0  2800.0  2800.00     N       1
                 16444-1-2024     1.0     1.0  4000.0  4000.00     N       1

[48108 rows x 6 columns]

In [71]:
no_recod = boletas02[boletas02['ArtCod'].str.len() != 9]
no_recod
#no_recod.to_excel('norecod.xlsx')

,RegCod,ArtCod,BolNum,UraChi,UreCan,ArtPhi,ArtPac,ArtCR,PerSem
13,01,1113032,60-1-2024,240.0,240.0,11.30,11.30,N,1
15,01,1113052,60-1-2024,12.0,12.0,19.30,19.30,N,1
16,01,1113053,60-1-2024,1.0,1.0,3.80,3.80,N,1
17,01,1113054,60-1-2024,1.0,1.0,5.20,5.20,N,1
18,01,1113055,60-1-2024,1.0,1.0,5.95,5.95,N,1
...,...,...,...,...,...,...,...,...,...
51306,08,11112074,16192-1-2024,1.0,1.0,18.00,18.00,N,3
51426,08,1199031,16398-1-2024,17.0,17.0,1.00,1.00,N,3
51444,08,1199031,16420-1-2024,17.0,17.0,1.00,1.00,N,3
51447,08,2110021,16420-1-2024,750.0,750.0,200.00,200.00,N,3


In [72]:
#no_recod[no_recod['ArtCod'] == '1113032']

In [73]:
#esta es la parte de los outliers
ignorar = False
if ignorar:
    ig = pd.read_excel(f'IGNORAR{anio}-{mes}.xlsx', dtype={f'ArtCod':str,f'RegCod':str})
    ig = ig.set_index(['RegCod','ArtCod','BolNum'])
else:
    ig = []

In [74]:
#Funcion para calcular las variaciones de cada cotización
def Variaciones(boleta, decada, outliers, ignorar = False):
    ''' 
    Esta función calcula las variaciones de la decada ingresada dado un arreglo de boletas y una decada
    
    Argumentos:
    boletas (DataFrame) : Un dataframe de las cotizaciones con las columnas RegCod, ArtCod, ArtPhi,ArtPac,UraChi,UreCan,ArtCR, PerSem y el multiíndice (RegCod, ArtCod)
    dec (int) : La década en la que se va a calcular el índice, 

    '''
    boletas = boleta.copy()
    if decada == 0:
        semana = [1,2,3]
    elif decada == 12:
        semana = [1,2]
    elif decada == 23:
        semana =[2,3]
    else:
        semana = [decada]

    boletas = boletas.loc[boletas['PerSem'].isin(semana)]
    
    Variacion = pd.DataFrame((boletas['ArtPac']*boletas['UraChi'])/(boletas['ArtPhi']*boletas['UreCan']), columns = ['Var'])
    Variacion = Variacion.replace([np.nan,0,np.inf],1)
    Variacion.loc[boletas['ArtCR'] == 'S'] = 1.0
    if ignorar:
            Variacion.loc[Variacion.index.isin(outliers.index), 'Var'] = 1.0
    return Variacion

In [75]:
variaciones = Variaciones(boletas, dec, ig, ignorar)
variaciones #variaciones del mes actual
#hasta aca estamos bien, las variacion si salen distintos valores

Var
RegCod ArtCod    BolNum                
01     011110101 1035-1-2024   1.000000
                 1104-1-2024   1.000000
                 1115-1-2024   1.000000
                 113-1-2024    1.021429
                 1175-1-2024   1.000000
...                                 ...
08     125110401 16025-1-2024  1.000000
                 16027-1-2024  1.000000
                 16052-1-2024  1.000000
                 16435-1-2024  1.000000
                 16444-1-2024  1.000000

[48108 rows x 1 columns]

In [76]:
#Función que selecciona las periodicidades de los artículos, convierte el código de la periodicidad al número de meses que significa.
def Periodos(connect):
        '''
        Llama las periodicidades de todos los artículos
        Args:
        connect (Connection) : conexión a la base de datos
        '''
        subquery = text(f"SELECT DISTINCT ArtCod FROM IPC2010_01_RN.dbo.IPCPH6 \
                WHERE PerAno = 2010 AND PerMes = 12 AND ArtCod != 091110301")
        query = text(f"SELECT ArtCod, \
                CASE \
                WHEN ArtPRC = 5 THEN  6\
                WHEN ArtPRC = 6 THEN  12\
                WHEN ArtPRC = 1 THEN  1\
                WHEN ArtPRC = 2 THEN  2\
                WHEN ArtPRC = 3 THEN  3\
                WHEN ArtPRC = 4 THEN  4\
                ELSE 1\
                END AS ArtPRC\
                FROM IPC2010_RN.dbo.IPC007 \
                WHERE ArtCod IN \
                ({subquery})\
                ORDER BY ArtCod")
        indice = 'ArtCod'
        return pd.read_sql(query, connect, index_col=indice)

In [77]:
#Función para calcular las variaciones acumuladas de los 441 artículos
def VarAcum(var,per, cod): #esta es la funcion que ellos llaman VarFlex
    ''' 
    Esta función calcula los índices relativos (variación acumulada) de los 441 artículos para todas las regiones y retorna un arreglo con índices (region, artículo). A los artículos que no fueron cotizados les asigna variación 1

    Args
    var (DataFrame) : un dataframe con las variaciones de las fuentes con multiíndice RegCod, ArtCod
    per (DataFrame) : dataframe con las periodicidades de los artículos con índice ArtCod
    cod (DataFrame) : un dataframe con los 441 códigos de los artículos
    '''

    agrupacion = var.groupby(level = ['RegCod','ArtCod']).agg(sp.gmean)
    periodicidad = agrupacion.join(per.astype(float), how = 'outer')
    raiz = pd.DataFrame(periodicidad['Var']**(1/(periodicidad['ArtPRC'])), columns = ['Var']).sort_index()
    arreglo = pd.DataFrame(['01','02','03','04','05','06','07','08'], columns= ['RegCod'])
    Codigos = cod.copy()
    Codigos['ArtCod'] = Codigos.index
    codigos2 = Codigos.merge(arreglo, how = 'cross')
    codigos2 = codigos2.set_index(['RegCod','ArtCod'])
    codigos2['Var'] = raiz['Var']
    codigos2 = codigos2.replace(np.nan, 1)
    return codigos2

In [78]:
#Llamamos las periodicidades y calculamos la variación de artículos
periodos = Periodos(connection)
Varac_prod = VarAcum(variaciones, periodos, codigos) #variacion enero
Varac_prod

,,Var
RegCod,ArtCod,
01,011110101,1.005875
02,011110101,1.000000
03,011110101,0.999304
04,011110101,1.026516
05,011110101,0.995836
...,...,...
04,125110501,1.000000
05,125110501,1.000000
06,125110501,1.000000


In [79]:
#Se agregan las variaciones del armado nacional
def FormacionNacional(indart):
    lista = ['071110101','071110102','071110103','072420201','073210101','083110101','083110202','083110301','083110302',\
                    '083110401','093210201','093310101','095110201','123110101','123110102','123110103','123110104','124110101','125110101','125110501']
    indices = indart.copy()
    for index in lista:
        try:
            indices.loc[(indart.index.get_level_values('ArtCod') == index),'Var'] = indart.loc['01', index]['Var']
        except KeyError:
            pass
    return indices

In [80]:
Varac_prod_01 = FormacionNacional(Varac_prod).sort_index() #variacion acumulada
Varac_prod_01

Var
RegCod ArtCod             
01     011110101  1.005875
       011110102  1.000000
       011110103  0.998505
       011120101  1.002611
       011120102  0.980704
...                    ...
08     125110101  1.000000
       125110201  1.000000
       125110301  1.000000
       125110401  1.000000
       125110501  1.000000

[3528 rows x 1 columns]

In [81]:
#Función para llamar la ponderación de cualquier categoría
def CatPon(cod : str, dic, connect):
    if cod == 'Art':
        subquery = text(f"SELECT DISTINCT {cod}Cod FROM IPC2010_01_RN.dbo.IPCPH6 \
                    WHERE PerAno = 2010 AND PerMes = 12 AND ArtCod != 091110301")
    else:
        subquery = text(f"SELECT DISTINCT {cod}Cod FROM IPC2010_01_RN.dbo.IPCM0{dic[cod]}")
    
    query = text(f'SELECT RegCod, {cod}Cod, {cod}Pon\
        FROM  IPC2010_RN.dbo.IPCP0{dic[cod]}\
        WHERE {cod}Cod IN ({subquery}) \
        ORDER BY RegCod, {cod}Cod')              
    return pd.read_sql(query, connect, index_col=['RegCod', cod+'Cod'])

In [82]:
#Función de cálculo de índices (relativos o absolutos)
def CatInd(cod :str, Varac_prod_01, pond, len = leng, codigo = codigos):
    pond = pond.copy()
    pond.columns = ['Pon']
    index = Varac_prod_01.copy()
    index.columns = ['Ind']
    pond[cod+'Cod'] = pond.index.get_level_values(1).map(lambda x: x[:len[cod]])
    pond = pond.set_index([pond.index.get_level_values(0), pond[cod+'Cod'], pond.index.get_level_values(1)]).drop(columns= [cod+'Cod'])
    pond[cod+'Pon'] = pond.groupby(level = ['RegCod', cod+'Cod']).sum()
    pond[cod+'Pon']=pond[cod+'Pon'].replace(0,1)
    cont = pd.DataFrame(index['Ind']*pond['Pon']/pond[cod+'Pon'], columns = [cod+'Ind'])
    cont[cod+'Cod'] = cont.index.get_level_values(1).map(lambda x: x[:len[cod]])
    cont = cont.set_index([cont.index.get_level_values(0), cont[cod+'Cod'], cont.index.get_level_values(1)]).drop(columns= [cod+'Cod'])
    catind = cont.groupby(level = ['RegCod', cod+'Cod']).sum()

    if cod == 'Gba':
        arreglo = pd.DataFrame(['01','02','03','04','05','06','07','08'], columns= ['RegCod'])
        cods = CodigosCat(cod, codigo)
        Codigos = cods.copy()
        Codigos[cod+'Cod'] = Codigos.index
        codigos2 = Codigos.merge(arreglo, how = 'cross')
        codigos2 = codigos2.set_index(['RegCod',cod+'Cod'])
        codigos2[cod+'Ind'] = catind[cod+'Ind']
        codigos2 = codigos2.replace(np.nan, 1)
        return codigos2
    else:
        return catind

In [83]:
CatPon('Gba', di, connection)

GbaPon
RegCod GbaCod          
00     0111101  0.63244
       0111201  0.17119
       0111202  1.33671
       0111203  0.93761
       0111301  2.61053
...                 ...
08     1251101  0.11386
       1251102  0.00713
       1251103  0.05303
       1251104  0.16740
       1251105  0.00000

[2511 rows x 1 columns]

In [84]:
pond = CatPon('Art', di, connection) #ponderaciones 
pond

ArtPon
RegCod ArtCod              
00     011110101   64.74764
       011110102   20.38296
       011110103   14.86940
       011120101   90.25644
       011120102    9.74356
...                     ...
08     125110101  100.00000
       125110201  100.00000
       125110301  100.00000
       125110401  100.00000
       125110501    0.00000

[3969 rows x 1 columns]

In [85]:
#Calculamos los índices relativos (variación) de gasto básico de las 8 regiones
Varprod02 = CatInd('Gba', Varac_prod_01, pond, leng)
Varprod02

,,GbaInd
RegCod,GbaCod,
01,0111101,1.002847
02,0111101,0.999845
03,0111101,0.999636
04,0111101,1.022518
05,0111101,0.997114
...,...,...
04,1251105,1.000000
05,1251105,0.000000
06,1251105,1.000000


In [86]:
#Función para extraer índices anteriores de cualquier categoría
def IndAnt(anio : int, mes : int, cod : str, conectar, dic = di, ano = False):
    if ano:
        mesant = mes
        anioant = anio - 1
    else:
        if mes == 1:
            anioant = anio -1
            mesant = 12
        else:
            anioant = anio
            mesant = mes-1

    query = text(f'SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_01_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_02_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_03_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_04_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_05_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_06_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_07_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_08_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        ORDER BY RegCod, {cod}Cod')
    return pd.read_sql(query, conectar, index_col=['RegCod',cod+'Cod'])

In [87]:
'''esto funciona solo para enero 2024'''
indant = IndAnt(anio, mes, 'Gba', connection) #estos serian los indices del mes anterior
indant

GbaInd
RegCod GbaCod             
01     0111101  177.796962
       0111201  244.137540
       0111202  238.104184
       0111203  150.948086
       0111301  175.782923
...                    ...
08     1241101   99.612750
       1251101  119.542238
       1251102  111.813829
       1251103  114.954121
       1251104  117.797094

[2185 rows x 1 columns]

In [88]:
'''esto seria solo para enero 2024'''
indanioant = IndAnt(anio, mes, 'Gba', connection, ano = True) #esto seria el indice del año anterior
indanioant

GbaInd
RegCod GbaCod             
01     0111101  159.608743
       0111201  222.889203
       0111202  229.021156
       0111203  140.140823
       0111301  179.111354
...                    ...
08     1241101   96.279220
       1251101  120.596493
       1251102  115.187051
       1251103  113.873077
       1251104  117.294604

[2185 rows x 1 columns]

In [89]:
indprod = pd.DataFrame(indant['GbaInd']*Varprod02['GbaInd'], columns= ['GbaInd']).replace(np.nan, 0)
indprod #este seria el indice de productos actuales

GbaInd
RegCod GbaCod             
01     0111101  178.303091
       0111201  243.911467
       0111202  232.956608
       0111203  151.170496
       0111301  173.572399
...                    ...
08     1251101  119.542238
       1251102  111.813829
       1251103  114.954121
       1251104  117.797094
       1251105    0.000000

[2232 rows x 1 columns]

In [90]:
#Función para llamar las ponderaciones de cada región
def RegPon(connect):
    query = text(f'SELECT RegCod, RegPon \
        FROM IPC2010_RN.dbo.IPC001 WHERE RegCod != 0')            
    return pd.read_sql(query, connect, index_col= 'RegCod')

In [91]:
#Función para calcular el índice por república
def IndRep(cod :str, index, regpond, indponde):
    indices = index.copy()
    regpon = regpond.copy()
    indpon = indponde.copy()
    indreg = indices.loc[(indices.index.get_level_values(0) != '00')]
    indpond = indpon.loc[(indpon.index.get_level_values(0) != '00')]
    reppon = pd.DataFrame(indpond[cod+'Pon']*regpon['RegPon'],columns = ['Pon'])
    sumas = reppon.groupby(level =cod+ 'Cod').sum()
    regcont = pd.DataFrame(indreg[cod+'Ind']*reppon['Pon']/sumas['Pon'],columns = [cod+'Ind'])
    indrep = regcont.groupby(level = [cod+'Cod']).sum()
    indrep['RegCod'] = '00'
    indrep = indrep.set_index([indrep['RegCod'], indrep.index]).drop(columns = ['RegCod'])
    indices.loc[indices.index.get_level_values(0) == '00'] = indrep
    return pd.concat([indrep, indreg])

In [92]:
#ponderaciones de la region a nivel republica
regpon = RegPon(connection)
regpon

,RegPon
RegCod,
01,37.34790
02,5.40404
03,7.43584
04,6.53469
05,10.96698
06,18.53248
07,9.66149
08,4.11658


In [93]:
#ponderaciones de los productos a nivel republica
gbapon = CatPon('Gba', di, connection)
gbapon

GbaPon
RegCod GbaCod          
00     0111101  0.63244
       0111201  0.17119
       0111202  1.33671
       0111203  0.93761
       0111301  2.61053
...                 ...
08     1251101  0.11386
       1251102  0.00713
       1251103  0.05303
       1251104  0.16740
       1251105  0.00000

[2511 rows x 1 columns]

In [94]:
#indice de productos del mes anterior a nivel nacional
indprod_repant = IndRep('Gba', indant, regpon, gbapon)
indprod_repant

GbaInd
RegCod GbaCod             
00     0111101  162.162129
       0111201  225.681685
       0111202  348.136965
       0111203  142.563944
       0111301  192.386176
...                    ...
08     1241101   99.612750
       1251101  119.542238
       1251102  111.813829
       1251103  114.954121
       1251104  117.797094

[2464 rows x 1 columns]

In [95]:
#indice prodcutos a nivel republica del año anterior
indprod_anioant = IndRep('Gba', indanioant, regpon, gbapon)
indprod_anioant

GbaInd
RegCod GbaCod             
00     0111101  150.611030
       0111201  207.558249
       0111202  341.562588
       0111203  136.887056
       0111301  191.108267
...                    ...
08     1241101   96.279220
       1251101  120.596493
       1251102  115.187051
       1251103  113.873077
       1251104  117.294604

[2464 rows x 1 columns]

In [96]:
#indice productos a nivel republica actual
indprod_rep = IndRep('Gba', indprod, regpon, gbapon)
indprod_rep

GbaInd
RegCod GbaCod             
00     0111101  162.174116
       0111201  225.867668
       0111202  349.099137
       0111203  142.609117
       0111301  190.087956
...                    ...
08     1251101  119.542238
       1251102  111.813829
       1251103  114.954121
       1251104  117.797094
       1251105    0.000000

[2511 rows x 1 columns]

In [97]:
indprod_rep01 = indprod_rep.reset_index()
indprod_rep01

,RegCod,GbaCod,GbaInd
0,00,0111101,162.174116
1,00,0111201,225.867668
2,00,0111202,349.099137
3,00,0111203,142.609117
4,00,0111301,190.087956
...,...,...,...
2506,08,1251101,119.542238
2507,08,1251102,111.813829
2508,08,1251103,114.954121
2509,08,1251104,117.797094


In [98]:
indprod_rep01.to_excel('indprod012024.xlsx')

In [99]:
#Nombres(indrep.loc[indrep['GbaInd'] != 0],'Gba').sort_values('GbaInd')

In [100]:
''' 
Aquí se pueden calcular los índices de cualquier categoría, sólo se debe cambiar el argumento Div (división) por el argumento Agr, Gru, Sub, Gba dependiendo del índice que se desee
'''
cat = 'Div'
inddiv = CatInd(cat, indprod_rep, gbapon)
pondiv = CatPon(cat, di, connection)
inddiv_rep = IndRep(cat, inddiv, regpon, pondiv)
inddiv_rep #indice de division a nivel republica actual

DivInd
RegCod DivCod            
00     01      280.230298
       02      129.673853
       03      120.247973
       04      129.131757
       05      131.623927
...                   ...
08     08      113.881410
       09      114.175054
       10      106.685441
       11      144.163055
       12      123.281345

[108 rows x 1 columns]

In [101]:
def IPC(index, pond):
    indices = index.copy()
    indices.columns = ['Ind']
    pon = pond.copy()
    pon.columns = ['Pon']
    return pd.DataFrame(indices['Ind']*pon['Pon']/100, columns = ['IPC']).groupby(level = ['RegCod']).sum()

In [102]:
'''esto seria el IPC actual'''
#IPC nacional actual
ipc_actual = IPC(inddiv_rep, pondiv)
ipc_actual

,IPC
RegCod,
00,174.213061
01,143.661861
02,268.983547
03,162.242677
04,243.107263
05,159.141537
06,161.446892
07,241.121030
08,179.832525


In [103]:
#IPC nacional mes pasado
ipc_anterior = IPC(indprod_repant, gbapon)
ipc_anterior

,IPC
RegCod,
00,174.337626
01,143.700289
02,267.153459
03,162.351535
04,247.007910
05,159.410544
06,161.115880
07,241.250594
08,178.995298


In [104]:
ipc_anioant = IPC(indprod_anioant, gbapon)
ipc_anioant

,IPC
RegCod,
00,167.976923
01,141.502683
02,251.132757
03,158.835808
04,234.628409
05,154.387647
06,153.795518
07,227.339056
08,170.438312


Variaciones

In [105]:
#variacion intermensual
(ipc_actual-ipc_anterior)*100/ipc_anterior

,IPC
RegCod,
00,-0.071451
01,-0.026742
02,0.685032
03,-0.067051
04,-1.579159
05,-0.168751
06,0.205450
07,-0.053705
08,0.467737


In [106]:
(ipc_actual-ipc_anioant)*100/ipc_anioant

,IPC
RegCod,
00,3.712497
01,1.525892
02,7.108109
03,2.144900
04,3.613737
05,3.079191
06,4.975031
07,6.062299
08,5.511796


Indidencias por division

In [107]:
def Incidencias(index, indexant, pond,ipcante):
    pon = pond.copy()
    pon.columns = ['Pon']
    indicesant = indexant.copy()
    indicesant.columns = ['Ind']
    indices = index.copy()
    indices.columns = ['Ind']
    return pd.DataFrame(pon['Pon']*(indices['Ind']-indicesant['Ind'])/ipcante['IPC'],columns= ['Incid'])

In [108]:
inddiv_repant = CatInd('Div', indprod_repant, gbapon)

In [109]:
incid_divrep = Incidencias(inddiv_rep, inddiv_repant, pondiv, ipc_anterior)
incid_divrep.loc[incid_divrep.index.get_level_values('RegCod') == '00']

Incid
RegCod DivCod          
00     01     -0.058702
       02     -0.000437
       03     -0.000452
       04     -0.028598
       05     -0.002470
       06     -0.000017
       07     -0.000820
       08     -0.007155
       09     -0.000223
       10      0.001779
       11      0.027430
       12     -0.001786

In [110]:
incid_divrep.sort_values('Incid')

Incid
RegCod DivCod          
04     01     -1.516135
05     01     -0.122713
03     04     -0.080823
07     01     -0.069289
05     04     -0.065687
...                 ...
       07      0.048005
01     11      0.072617
06     01      0.187179
08     01      0.516907
02     01      0.679541

[108 rows x 1 columns]

Incidencia productos

In [111]:
incid_prodrep = Incidencias(indprod_rep, indprod_repant, gbapon, ipc_anterior)
incid_prodrep.loc[incid_prodrep.index.get_level_values('RegCod') == '00']

Incid
RegCod GbaCod           
00     0111101  0.000043
       0111201  0.000183
       0111202  0.007377
       0111203  0.000243
       0111301 -0.034414
...                  ...
       1251101  0.000000
       1251102  0.000000
       1251103  0.000000
       1251104  0.000011
       1251105  0.000000

[279 rows x 1 columns]

In [112]:
incid_prodrep.loc[incid_prodrep['Incid'].notnull()]

Incid
RegCod GbaCod           
00     0111101  0.000043
       0111201  0.000183
       0111202  0.007377
       0111203  0.000243
       0111301 -0.034414
...                  ...
08     1241101  0.000000
       1251101  0.000000
       1251102  0.000000
       1251103  0.000000
       1251104  0.000000

[2464 rows x 1 columns]

incidencias region

In [113]:
ipc_actualreg = ipc_actual.loc[ipc_actual.index != '00']
ipc_antreg = ipc_anterior.loc[ipc_anterior.index != '00']
ipc_anttotal = ipc_anterior.loc['00','IPC']

In [114]:
#incid_reg = 
pd.DataFrame(regpon['RegPon']*(ipc_actualreg['IPC']-ipc_antreg['IPC'])/ipc_anttotal, columns = ['Incid'])
#incid_reg

,Incid
RegCod,
01,-0.008232
02,0.056728
03,-0.004643
04,-0.146208
05,-0.016922
06,0.035187
07,-0.007180
08,0.019769
